In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from clust_huge_amp import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="sklearn.utils.extmath")


## Running the model for prediction tasks


Here we demonstrate model initialization for a model that will ultimately use data only at enrollment for prediction tasks. We initialize the model as in for retrospective tasks, but we modify the Censoring matrix E such that an individual's event times are capped at the minimum of event time or age of recruitment, whicever comes first.


In [ ]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import

def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_matrix.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()


In [ ]:
from clust_huge_amp import *
# Subset the data
#Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=0, end_index=10000)

Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=20000, end_index=30000)

torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Initialize model with subsetted data

del Y

In [ ]:
# Load references (signatures only, no healthy)
refs = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/reference_trajectories.pt')
signature_refs = refs['signature_refs']
# When initializing the model:


In [ ]:
import pandas as pd
fh_processed=pd.read_csv('/Users/sarahurbut/Library/Cloudstorage/Dropbox/baselinagefamh.csv')
len(fh_processed)

You can see that we take the event times and cap them at the enrollment time or the age of event, whichever occurs first, so if someone has an event prior to enrollment that age (index) is preserved but there is no `leakage of inofrmaiton from the future

In [ ]:
# Create enrollment-time-aware E matrix
E_enrollment = E_100k.clone()  # Start with the original censoring times

# For each patient, cap event/censoring times at their enrollment time
pce_df_subset = fh_processed.iloc[20000:30000].reset_index(drop=True)
#pce_df_subset = fh_processed.iloc[0:10000].reset_index(drop=True)
for patient_idx, row in enumerate(pce_df_subset.itertuples()):
    if patient_idx >= E_enrollment.shape[0]:  # Make sure we don't go out of bounds
        break
        
    # Calculate enrollment time (age at enrollment - 30)
    enroll_time = int(row.age - 30)
    
    if enroll_time >= 0:  # Make sure enrollment time is valid
        # Cap all disease times to be at most the enrollment time
        # This is the key step: min(original_event_time, enrollment_time)
        E_enrollment[patient_idx, :] = torch.minimum(
            E_enrollment[patient_idx, :], 
            torch.full_like(E_enrollment[patient_idx, :], enroll_time)
        )

In [ ]:
for person_idx in range(E_100k.shape[0]):
    enroll_time = int(pce_df_subset.iloc[person_idx]['age'] - 30)
    for disease_idx in range(E_100k.shape[1]):
        event_time = E_100k[person_idx, disease_idx].item()
        if event_time < enroll_time:
            capped_time = E_enrollment[person_idx, disease_idx].item()
            print(f"Person {person_idx}, Disease {disease_idx}:")
            print(f"  Event time: {event_time}")
            print(f"  Enrollment time: {enroll_time}")
            print(f"  Capped time: {capped_time}")
            if capped_time != event_time:
                print("  WARNING: Event time before enrollment was incorrectly capped!")
            else:
                print("  OK: Event time before enrollment is unchanged.")
            break  # Stop after first found
    else:
        continue
    break  # Stop after first found

In [ ]:

G_with_sex = np.column_stack([G_100k, sex])  # sex should be numeric (e.g., 0/1)
# N

In [ ]:
G_with_sex.shape

In [ ]:


model = AladynSurvivalFixedKernelsAvgLoss_clust_logitInit_psitest(
    N=Y_100k.shape[0], 
    D=Y_100k.shape[1], 
    T=Y_100k.shape[2], 
    K=20,
    P=G_with_sex.shape[1],
    init_sd_scaler=1e-1,
    G=G_with_sex, 
    Y=Y_100k,
    genetic_scale=1,
    W=0.0001,
    R=0,
    prevalence_t=essentials['prevalence_t'],
    signature_references=signature_refs,  # Only pass signature refs
    healthy_reference=True,  # Explicitly set to None
    disease_names=essentials['disease_names']
)

torch.manual_seed(0)
np.random.seed(0)
# Initialize with psi and clusters


import cProfile
import pstats
from pstats import SortKey

# Now in your batch run, load and verify:
initial_psi = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/initial_psi_400k.pt')
initial_clusters = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/initial_clusters_400k.pt')

model.initialize_params(true_psi=initial_psi)
model.clusters = initial_clusters
# Verify clusters match
clusters_match = np.array_equal(initial_clusters, model.clusters)
print(f"\nClusters match exactly: {clusters_match}")


In [ ]:
print(model.G.shape)
print(model.gamma.shape)

Fit the model

In [ ]:
import cProfile
import pstats
from pstats import SortKey

# For the healthtoo version
profiler = cProfile.Profile()
profiler.enable()

history_new = model.fit(E_enrollment, num_epochs=200, learning_rate=1e-1, lambda_reg=1e-2)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats(SortKey.CUMULATIVE)
stats.print_stats(20)

In [ ]:
from utils import *
plot_training_evolution(history_new)

In [ ]:
# Get predictions (pi) from the model
with torch.no_grad():
    pi, _, _ = model.forward()  # pi shape: (N, D, T)



Y_test = model.Y
test_start=20000
test_end=30000
FH_test = fh_processed.iloc[test_start:test_end].reset_index(drop=True)
# For each subject, get their enrollment age (from your covariate DataFrame)
# Suppose FH_train is your DataFrame for the training set (reset index to match order)
aladynoulli_1yr_risk_test = []
for i, row in FH_test.iterrows():
    enroll_time = int(row['age'] - 30)
    # Choose the disease indices you want (e.g., ASCVD indices)
    # For a single disease: d_idx = ...
    # For a group: d_indices = [...]
    # Example for ASCVD:
    d_indices = [111, 112, 113, 114, 115, 116]  # adjust as needed
    if enroll_time < 0 or enroll_time >= pi.shape[2]:
        aladynoulli_1yr_risk_train.append(np.nan)
        continue
    # 1-year risk for ASCVD group at enrollment
    pi_diseases = pi[i, d_indices, enroll_time]
    yearly_risk = 1 - torch.prod(1 - pi_diseases)
    aladynoulli_1yr_risk_test.append(yearly_risk.item())

import numpy as np
import torch
ascvd_indices=[111,112,113,114,115,116]
cases = []
controls = []
for i in range(len(FH_test)):
    age = FH_test.iloc[i]['age']
    t_enroll = int(age - 30)
    if t_enroll < 0 or t_enroll >= Y_test.shape[2]:
        continue
    end_time = min(t_enroll + 10, Y_test.shape[2])
    had_event = 0
    for d_idx in ascvd_indices:
        if torch.any(Y_test[i, d_idx, t_enroll:end_time] > 0):
            had_event = 1
            break
    if had_event:
        cases.append(aladynoulli_1yr_risk_test[i])
    else:
        controls.append(aladynoulli_1yr_risk_test[i])
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.histplot(cases, color='red', label='ASCVD Case', kde=True, stat='density', bins=30, alpha=0.6)
sns.histplot(controls, color='blue', label='Control', kde=True, stat='density', bins=30, alpha=0.6)
plt.xlabel('Aladynoulli 1-year risk')
plt.ylabel('Density')
plt.title('Distribution of Aladynoulli 1-year risk for ASCVD Cases vs Controls')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:

# Save model
W=0.0001
torch.save({
    'model_state_dict': model.state_dict(),
    'E':E_enrollment,
    'phi': model.phi,
    'Y': model.Y,
    'prevalence_t': model.prevalence_t,
    'logit_prevalence_t': model.logit_prev_t,
    'G': model.G,
}, f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_model_W{W}_jointphi_sexspecific_20000_30000.pt')

In [ ]:

plot_theta_differences(model, diseases=[112,67,127,10,17,114], signatures=[5,7,0,17,19,5])